Find the leading scorer by zone on the court. Plot the zones and players' names.

In [5]:
import sys
import numpy as np
from matplotlib import pyplot as plt
from euroleague_api.shot_data import ShotData

sys.path.append("../utils/")
from shot_chart_plots import plot_scatter, plot_zones, plot_leading_scorers_by_zone, fg_perc_hex_heatmap  #, plot_scatter_single_df
# %matplotlib

plt.close("all")

In [6]:
def get_leading_scorer_by_zone(df, zone_col="ZONE"):
    """Calculate the leading scorers by zone

    Args:
        df (pd.DataFrame): The dataframe of the shooting data
        zone_col (str): The column name of the zone data.

    Returns:
        pd.DataFrame: A dataframe with the leading scorers by zone and their points
            scored in that zone
    """
    top_players_by_zone_df = df.groupby([zone_col, "PLAYER"])["POINTS"].sum().reset_index()
    pidx = top_players_by_zone_df.groupby(zone_col)['POINTS'].idxmax()
    top_players = top_players_by_zone_df.loc[pidx]
    # top_players.rename(columns={zone_col: "ZONE"}, inplace=True)
    return top_players


def corner_three_zones(df, corner_three_y=147.5):
    """
    Split the two 3pt zones into corner three and wing three.

    Args:
        df (pd.DataFrame): The dataframe of the shooting data.
        corner_three_y (float, optional): The y coord defining the span of corner three.
            Defaults to 147.5.

    Returns:
        pd.DataFrame: The shooting data dataframe with an extra column the corrected
            zones with corder threes
    """
    df["ZONE_C"] = df["ZONE"].copy()
    corner_three_mask = df["ZONE"].isin(["H", "I"]) & (df["COORD_Y"] <= corner_three_y)
    df.loc[corner_three_mask, "ZONE_C"] += "-Corner3"
    return df


def hexagon_zones(df, gridsize=10):
    x = df["COORD_X"]
    y = df["COORD_Y"]
    z = np.where(df["POINTS"] > 0, 1, 0)

    # cmap = plt.cm.Oranges
    _, ax = plt.subplots()
    hb = ax.hexbin(x=x, y=y, C=z, gridsize=gridsize,
                   extent=[-800, 800, -200, 1300],
                   alpha=0.8, vmin=0., vmax=1.)
    # plt.close()

    offsets = hb.get_offsets()
    # Create an empty array to store the indices of the bins
    indices = []

    # Loop through each point and find which bin it belongs to
    for x_, y_ in zip(x, y):
        dist = np.linalg.norm(offsets - np.array([x_, y_]), axis=1)
        closest_bin = np.argmin(dist)
        indices.append(hash((offsets[closest_bin][0], offsets[closest_bin][1])))

    df["ZONE_HEX"] = indices
    return df

In [7]:
season = 2024
competition = "E"

In [ ]:
shotdata = ShotData(competition)
shot_df = shotdata.get_game_shot_data_single_season(season)

# Find leading scorers by zone

In [9]:
round_ = None  # Set to None if you want all rounds of the season

In [10]:
shot_df = corner_three_zones(shot_df)
# shot_df = hexagon_zones(shot_df, gridsize=3)
# fg_perc_hex_heatmap(shot_df, gridsize=4)

In [ ]:
if round_ is not None:
   top_players = get_leading_scorer_by_zone(shot_df[(shot_df["Round"] == round_)], zone_col="ZONE_C")
else:
    top_players = get_leading_scorer_by_zone(shot_df, zone_col="ZONE_C")
top_players

In [ ]:
plot_leading_scorers_by_zone(shot_df, top_players, zone_col="ZONE_C")

# Player Shooting Chart

In [39]:
player_name = 'NEDOVIC, NEMANJA'
round_ = 1  # Set to None if you want all rounds of the season

In [ ]:
player_mask = (shot_df["PLAYER"] == player_name)
if round_ is not None:
     player_mask &= (shot_df["Round"] == round_)

player_shot_df = shot_df[shot_df["PLAYER"] == player_name]
palyer_missed_mask = player_shot_df["ID_ACTION"].str.contains("FGA")
player_made_mask = player_shot_df["ID_ACTION"].str.contains("FGM")

plot_scatter(player_shot_df[player_made_mask], player_shot_df[palyer_missed_mask], title=f'{player_name} - Shot Chart', color="red")

# Teams' shooting chart

In [36]:
from euroleague_api.team_stats import TeamStats

teamstats = TeamStats("E")
team_info = teamstats.get_team_stats_single_season(endpoint="traditional", season=2024, phase_type_code=None, statistic_mode="PerGame")
code_to_name = team_info[["team.code", "team.name"]].set_index("team.code").to_dict()["team.name"]

In [31]:
team_abbr = "PAN"
round_ = 2  # Set to None if you want all rounds of the season

In [ ]:
team_mask = (shot_df["TEAM"] == team_abbr)
if round_ is not None:
     team_mask &= (shot_df["Round"] == round_)

team_shot_df = shot_df[(shot_df["TEAM"] == team_abbr) & (shot_df["Round"] == round_)]
team_missed_mask = team_shot_df["ID_ACTION"].str.contains("FGA")
team_made_mask = team_shot_df["ID_ACTION"].str.contains("FGM")

plot_scatter(team_shot_df[team_made_mask], team_shot_df[team_missed_mask], title=f'{code_to_name[team_abbr]} - Shot Chart', color="red")

## Plot shooting charts of all teams

In [18]:
round_ = 2  # Set to None if you want all rounds of the season

In [ ]:
for team_abbr in shot_df["TEAM"].unique():
    team_mask = (shot_df["TEAM"] == team_abbr)
    if round_ is not None:
        team_mask &= (shot_df["Round"] == round_)
    team_shot_df = shot_df[team_mask]
    team_missed_mask = team_shot_df["ID_ACTION"].str.contains("FGA")
    team_made_mask = team_shot_df["ID_ACTION"].str.contains("FGM")
    plot_scatter(team_shot_df[team_made_mask], team_shot_df[team_missed_mask], title=f'{code_to_name[team_abbr]} - Shot Chart', color="red")